# Calculating similarity between datasets (DICE)
1. Import neccesary libraries 
2. Upload Datasets
3. Run Similarity algorithm

In [ ]:
#Import Libraries
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
import numpy as np
import json

In [ ]:
#Upload datasets
green = []
skillspan = []
sayfullina = []
path = "./Data/Supervised"

#Green (g)
with open(path+'/Green/total_train.json', 'r') as f:
    for line in f:
        green.append(json.loads(line.strip()))

#SkillSpan (ss)
with open(path+'/SkillSpan/total_train.json', 'r') as f:
    for line in f:
        skillspan.append(json.loads(line.strip()))

In [ ]:
#Function to extract all spans 
def extract_relevant_spans(data):
    relevant_spans = []
    
    for entry in data:
        tokens = entry['tokens']
        tags = entry['tags_skill']
        
        i = 0
        while i < len(tags):
            if tags[i] == 'B':
                # Start a new span
                span_tokens = [tokens[i]]
                i += 1
                
                # Collect following 'I' tags
                while i < len(tags) and tags[i] == 'I':
                    span_tokens.append(tokens[i])
                    i += 1
                
                # Append the span to the list as a single string
                relevant_spans.append(' '.join(span_tokens))
            else:
                i += 1
    
    # Confirm that all elements are strings
    assert all(isinstance(span, str) for span in relevant_spans), "Not all spans are strings"

    return relevant_spans

In [ ]:
#Extracting relevant spans 
relevant_spans_g = extract_relevant_spans(green)
relevant_spans_ss = extract_relevant_spans(skillspan)

In [ ]:
#Mesure similarity across spans 

tokenized_spans_ss = [set(word_tokenize(span.lower())) for span in relevant_spans_ss]
tokenized_spans_g = [set(word_tokenize(span.lower())) for span in relevant_spans_g]

#Function to calculate the Dice coefficient
def dice_coefficient(set1, set2):
    intersection = len(set1.intersection(set2))
    return 2 * intersection / (len(set1) + len(set2))

#Initialize a matrix to store the Dice coefficient for each pair of spans
dice_matrix = np.zeros((len(tokenized_spans_ss), len(tokenized_spans_g)))

#Calculate the Dice coefficient for each pair of spans between the two datasets
for i, span_ss in enumerate(tokenized_spans_ss):
    for j, span_green in enumerate(tokenized_spans_g):
        dice_matrix[i, j] = dice_coefficient(span_ss, span_green)

print("Pairwise Dice Similarity Matrix:")
print(dice_matrix)